In [19]:
import random
import string

from Crypto.Random.random import shuffle
from secrets import randbelow


class BitOfShuffling:
    def __init__(self, key_length):
        self.perm = [x for x in range(key_length)]
        shuffle(self.perm)

    def shuffle_int(self, input_int: int):
        shuffled_int = 0
        for x in range(len(self.perm)):
            shuffled_int |= ((input_int >> x) & 1) << self.perm[x]
        return shuffled_int

    def shuffle_bytes(self, input_bytes):
        return self.shuffle_int(int.from_bytes(input_bytes, 'big'))


def rand_string(length):
    return ''.join(
        random.choices(string.digits + string.ascii_letters + r"""!"#$%&'()*+,-./:;<=>?@[\]^_`|~""", k=length))


def pad_flag(flag, length):
    pad_size = length - len(flag)
    if pad_size == 0:
        return flag
    left_size = randbelow(pad_size)
    right_size = pad_size - left_size
    return rand_string(left_size) + flag + rand_string(right_size)


KEY_LENGTH = 128
trials = 10
shuffler = BitOfShuffling(1024)

In [20]:
from Crypto.Util.number import *

In [21]:
ans=[]
for i in range(1,11):
    ans.append((2**i)*((2**(10-i))*'1' + (2**(10-i))*'0'))

In [22]:
sans =[]
for i in range(10):
    t1=shuffler.shuffle_bytes(long_to_bytes(int(ans[i],2)))
    sans.append(bin(t1)[2:])

In [23]:
for i in range(len(sans)):
    sans[i]=sans[i].rjust(1024,'0')

In [24]:
from pwn import *

In [25]:
r = remote("shuffling-as-a-service.ctf.umasscybersec.org","1337")

[x] Opening connection to shuffling-as-a-service.ctf.umasscybersec.org on port 1337
[x] Opening connection to shuffling-as-a-service.ctf.umasscybersec.org on port 1337: Trying 34.75.200.137
[+] Opening connection to shuffling-as-a-service.ctf.umasscybersec.org on port 1337: Done


In [26]:
r.recvuntil("string:")
r.recvline()
oint=int(r.recvline().strip().decode(),16)

/tmp/ipykernel_9360/4171539603.py:1: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  r.recvuntil("string:")


In [27]:
x=[]
print(r.recv())
for i in range(10):
    r.sendline(hex(int(ans[i],2))[2:])
    y1=r.recv()
    x.append(y1[:y1.index(b"\n")])

b'You too can shuffle your hexed bits with our 10 free trials!\nInput 1:'


/tmp/ipykernel_9360/2558109166.py:4: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  r.sendline(hex(int(ans[i],2))[2:])


In [28]:
sans=[]
for i in x:
    sans.append(bin(int(i.decode(),16))[2:])


In [29]:
for i in range(len(sans)):
    sans[i]=sans[i].rjust(1024,'0')

In [30]:
ans

['...']

In [37]:
sans

['11100000011111100001011111101110000100100110011011010011000000111000111100010111010111011100001001110001101100111000100100110101101110010100011100111001001000100010000010011101011001010100111111000011001110001111000011000110010011111010010110101000001001101111011010101110001101000110111001000000011110010100110010001100000111101100010110111101100011111000000010110001110100000001010011011011011111010010100101011111000000001111011100110001101111100100000111110001100100100110110100011001100100110101111010100101010010010011010000100000011001000101011010000111111101000011011010100101100111011110111101001000001110010011010101001010010111011111100100001110011110011110011000101011010111000110010100100010101011011100110111100001101001001010010110111111011001010011101001101010000111010010000100101001011100111110100111011010101001000001000000011111011001110010101100011010100010101101010001101011101010100100011101110100010010100011111001101111101101010010001010100011100011110100011110010111000101

In [31]:
f1=[]
for i in range(1024):
    t2=''
    for j in range(10):
        t2+=sans[j][i]
    f1.append(int(t2,2))

In [33]:
h1=['0' for i in range(1024)]

In [34]:
oint = bin(oint)[2:]
oint=oint.rjust(1024,'0')

In [35]:
for i in range(len(f1)):
    h1[f1[i]] = oint[i]

In [36]:
long_to_bytes(int(''.join(h1)[::-1],2))

b'D`O>JK&*_B5Tz7n|~F6#`Eb2F%]Q?;djD6)yH<UMASS{6Huff3d_2_b1t5}gJY*]h"\'blx%t$PC|~bl^BQ?vrp@4f(\',D4KTIZRLDF/tya>zlXfMgmUu^7+NVEES=h9&'